# Assignment 1

Name        : Chia Sin Liew   
Last edited : February 2nd, 2022 

The goal of this assignment is to study the K-Nearest Neighbors(K-N) classifier model.

- **Part A**: Model optimization via feature selection & varying threshold
- **Part B**: Understanding the curse of dimensionality & the fundamental limitation of the K-NN model

## Part A: Classification of Structured Data

### **Dataset**:
The *KNNDataset.csv* has 250 samples & 300 features.

  - Feature columns: column '0' to column '299'.
  - Target column: column 'target'.


### **Pre-Processing**:

- Load the CSV file as a Pandas DataFrame object


In [ ]:
import pandas as pd

df = pd.read_csv("KNNDataset.csv")
df

  - Create a data frame object for the features and another data frame object for the target

In [ ]:
# Y for target
Y = df.target.to_frame()
# X for features
X = df.drop(columns=["id", "target"])

print("\nData frame for features(X): \n", X.iloc[:6, :6])
print("\nDimension of X: ", X.shape)
print("\nData frame for target(Y): \n", Y)

- Convert the above two data frame objects into two NumPy arrays (you may use te NumPy "asarray" function).

In [ ]:
import numpy as np

X = np.asarray(X)
X

In [ ]:
Y = np.asarray(Y)
Y

- Convert the target array type into "int"

In [ ]:
print("X data type: ", X.dtype)
print("Y data type: ", Y.dtype)

Y = Y.astype(int)
print("\nY data type after conversion: ", Y.dtype)

- Partition the dataset into training & test subsets: 80% training & 20% test(you may use Scikit_learn's train_test_split() function)